# XGBoost模型调优

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

#####################################################
#####  Load data
#####################################################

feature_file = pd.read_csv("./DataHousePricePrediction/train.csv")

x = []# feature data
y = []# label
for index in feature_file.index.values:
    #print('index', index)
    #print(feature_file.values[0])
    #print(feature_file.ix[index].values) 
    x.append(feature_file.values[index][2: -1]) # get feature data from original csv
    y.append(feature_file.values[index][1])     # get label from original csv
   
x, y = np.array(x), np.array(y)
# Divide train data and valid data
X_train,X_valid,y_train,y_valid = train_test_split(x,y,test_size=0.2,random_state=12345)

#####################################################
##### Create XGBoost model
#####################################################

xgb = XGBRegressor(learning_rate =0.1,
                   n_estimators=150, 
                   max_depth=5,
                   min_child_weight=1,
                   gamma=0,
                   subsample=0.8,
                   colsample_bytree=0.8,
                   objective= 'reg:squarederror',
                   reg_alpha= 0,
                   reg_lambda= 1,
                   nthread=4,
                   scale_pos_weight=1,
                   seed=27)


#####################################################
##### Grid search optimal parameters
#####################################################
from sklearn.model_selection import GridSearchCV
#Need to research
#research_one: n_epoch
#research_two: max_depth
param_test1 = {
    'min_child_weight': [1, 2, 3],
    'max_depth':[3, 4, 5, 6, 7]
    }

xgb_res = GridSearchCV(estimator = xgb, 
                       param_grid = param_test1, 
                       n_jobs=4, 
                       cv=5)

xgb_res.fit(X_train, y_train)
print('max_depth_min_child_weight')
print('gsearch1.grid_scores_', xgb_res.cv_results_)
print('gsearch1.best_params_', xgb_res.best_params_)
print('gsearch1.best_score_', xgb_res.best_score_)

max_depth_min_child_weight
gsearch1.grid_scores_ {'mean_fit_time': array([1.22470727, 0.15125628, 0.13625121, 0.18264093, 0.16463671,
       0.17036152, 0.22602458, 0.20644631, 0.19484382, 0.28498874,
       0.25549579, 0.22841611]), 'std_fit_time': array([0.51665587, 0.01402901, 0.01155262, 0.00560831, 0.00287121,
       0.0032931 , 0.00430299, 0.00338323, 0.0041674 , 0.00974867,
       0.00517105, 0.0056183 ]), 'mean_score_time': array([0.00300069, 0.00330224, 0.00260034, 0.00300069, 0.00340099,
       0.00320067, 0.00320067, 0.00300059, 0.00300069, 0.00390472,
       0.00380077, 0.00320067]), 'std_score_time': array([6.32635950e-04, 1.16882113e-03, 4.89706807e-04, 1.78416128e-07,
       4.90018230e-04, 4.00114187e-04, 3.99994918e-04, 1.90734863e-07,
       1.78416128e-07, 1.92237336e-04, 4.00018735e-04, 3.99994889e-04]), 'param_max_depth': masked_array(data=[5, 5, 5, 7, 7, 7, 9, 9, 9, 11, 11, 11],
             mask=[False, False, False, False, False, False, False, False,
           